In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

Empezamos preparando el dataset y la variable k que nos indica la cantidad de features finales que vamos a elegir

In [2]:
k = 2
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Obtenemos una copia del dataset que vamos a utilizar, para éste caso elegimos las imagenes con las que realiza el entrenamiento de el dataset fashion_mnist.

In [3]:
data_o = train_images.copy()
data = train_images.copy()

Procedemos con los pasos detallados para realizar PCA con la librería numpy (np).

1) Normalizar/escalar los datos
restar media
dividir por dest. estándar 

In [4]:
data = data.reshape(60000,784)
data_o = data_o.reshape(60000,784)

In [5]:
data = (data - np.mean(data,axis=0) ) / np.std(data,axis = 0) 

In [6]:
data.shape

(60000, 784)

2) Calcular la matriz de coovarianza y calcular: 
Vectores propios
Valores propios de cada vector propio 

In [7]:
cov_M = np.cov(data.T) / data.shape[0]

In [8]:
v, w = np.linalg.eig(cov_M)
idx = v.argsort()[::-1] 
v = v[idx] 
w = w[:,idx] 

3) Hacer una proyección ortogonal de los datos normalizados sobre el sub-espacio principal(definido por los eigen-vectors asociados a los eigen-values mayores) 

In [9]:
data_PCA = data.dot(w[:, :k])
data_PCA.shape

(60000, 2)

In [10]:
tmp = datetime.now()
data_PCA_TSNE = TSNE(n_components=k).fit_transform(data)
tmp-datetime.now()

datetime.timedelta(days=-1, seconds=76147, microseconds=482991)

In [11]:
data_PCA_TSNE.shape

(60000, 2)

In [12]:
data_PCA_TSNE_pd = pd.DataFrame(data_PCA_TSNE)
data_PCA_TSNE_pd.head()

,0,1
0,25.029808,38.470676
1,0.596339,35.355629
2,-8.947552,-13.391912
3,-14.046834,-13.781363
4,-18.721214,-25.006678


In [13]:
data_PCA_pd = pd.DataFrame(data_PCA)
data_PCA_pd.head()

,0,1
0,-0.801175,-20.866749
1,17.074206,4.968512
2,-9.601641,12.296101
3,-0.311206,11.576765
4,8.963833,12.867738


Comprobamos que nuestra implementación de PCA no difiera significativamente de la obtenida por PCA.

In [14]:
data_PCA_sk = PCA(n_components=k).fit_transform(data)

In [15]:
data_PCA_sk = pd.DataFrame(data_PCA_sk)
data_PCA_sk.head()

,0,1
0,-0.801175,20.866749
1,17.074206,-4.968513
2,-9.601641,-12.296101
3,-0.311206,-11.576765
4,8.963833,-12.867738


Nuestra implementación de PCA genera resultados equivalentes, salvo por signos en algunos casos, de la implementación de PCA de sklearn.

Donde podemos ver que los valores de ambas representaciones en 2 dimensiones son diferentes.

In [16]:
np_dataset = np.array(data_PCA_pd).astype('float')
kmeans = KMeans(n_clusters = 10,random_state = 0).fit(np_dataset)
kmeans.labels_

array([0, 9, 1, ..., 5, 1, 7])

In [17]:
id_labels = range(10)
str_labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

dict_labels = dict(zip(id_labels,str_labels))
dict_labels

{0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'}

In [18]:
pd.crosstab(np.array(kmeans.labels_), np.array([dict_labels[x] for x in train_labels]), rownames = ['KMeans'], colnames = ['Real'])

Real,Ankle boot,Bag,Coat,Dress,Pullover,Sandal,Shirt,Sneaker,T-shirt/top,Trouser
KMeans,,,,,,,,,,
0,882,2100,22,0,46,9,67,7,18,5
1,8,295,384,1516,607,39,1080,0,1122,1357
2,179,416,1745,41,1540,9,1106,1,211,39
3,1235,472,8,1,24,1679,44,2853,8,0
4,3496,1378,8,1,23,219,35,312,7,1
5,1,15,242,2773,18,1,342,0,1028,4338
6,2,160,1859,21,2123,0,1131,0,314,23
7,73,124,54,3,117,3971,135,2825,35,2
8,124,1021,775,141,1308,72,1256,2,443,45


In [19]:
np_dataset_TSNE = np.array(data_PCA_TSNE).astype('float')
kmeans_TSNE = KMeans(n_clusters = 10,random_state = 0).fit(np_dataset_TSNE)
pd.crosstab(np.array(kmeans_TSNE.labels_), np.array([dict_labels[x] for x in train_labels]), rownames = ['KMeans'], colnames = ['Real'])

Real,Ankle boot,Bag,Coat,Dress,Pullover,Sandal,Shirt,Sneaker,T-shirt/top,Trouser
KMeans,,,,,,,,,,
0,4,66,629,459,1329,42,1713,1,1492,139
1,1904,25,0,3,0,1828,4,1584,0,0
2,0,503,5,452,5,0,6,0,11,5568
3,42,190,54,69,421,62,1242,0,3467,15
4,0,38,2089,7,2347,0,1330,0,23,15
5,1,37,2417,303,1793,0,964,0,58,10
6,18,33,0,0,2,2278,0,4332,2,0
7,0,45,789,4562,76,0,519,0,743,227
8,4031,11,0,0,0,1352,1,77,7,0


## Conclusiones

- Realizando KMeans tanto sobre el conjunto al que nosotros le hacemos PCA como al que creado por t-sne vemos que en éste conjunto, el t-sne realiza una mejor separación, sin embargo no hacen una separación perfecta.
- La implementación de PCA tanto manual como creada con el apoyo de la librería obtuvieron resultados que para fines prácticos son equivalentes.
- El tiempo de ejecucción de t-sne fue de casi 3 horas mientras que las implementaciones de PCA son de pocos minutos, pero ese tiempo se ve reflejado en su efectividad en la separación.